In [26]:
import scipy.stats as scs
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import nltk
from datetime import date, datetime, timedelta
from collections import Counter
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')
%matplotlib inline

In [54]:
df = pd.read_csv('data/data_nobadrows.csv')

In [15]:
### Convert currencies

df = pd.read_csv('data/data_nobadrows.csv')
keep_cols = [x for x in df.columns if 'Unnamed' not in x]
df = df[keep_cols]
df.loc[:,'goal'] = df.loc[:,'goal'].astype(int)
df.loc[:,'pledged'] = df.loc[:,'pledged'].astype(int)
df.loc[:,'backers'] = df.loc[:,'backers'].astype(int)

df['launched'] = df['launched'].apply(get_date)
df['deadline'] = df['deadline'].apply(get_date)

get_nasty = df[np.isnan(df['usd pledged'])]
dates = get_nasty['launched'].unique()

currencys = ','.join(get_nasty['currency'].unique())

exchange_rates = pd.read_csv('data/exchange.csv').set_index('date')

get_nasty_ = get_nasty[get_nasty['currency'] != 'DKK']

get_nasty_['usd pledged'] = get_nasty_.T.apply(lambda x: x['pledged'] * \
                 exchange_rates.loc[str(x['launched']),'USD'] / \
                 exchange_rates.loc[str(x['launched']),x['currency']]) 

df.loc[get_nasty_.index, 'usd pledged'] = get_nasty['usd pledged']

df_ = df[~np.isnan(df['usd pledged'])]

df_.to_csv('data/data_with_usd.csv')

/Users/travis/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [65]:
### add parts of speech tagging
df = pd.read_csv('data/data_with_usd.csv')
names = df['name']
df.loc[:,'name'] = df.loc[:,'name'].fillna('None')
lett_count = names.apply(lambda x: len(re.sub('[^a-z]', '', x.lower())))
word_count = names.apply(lambda x: len(re.findall("[a-z']+", x.lower())))
punc_count = names.apply(lambda x: len(re.findall("[\p{P}\d]", x))) / lett_count
caps_count = names.apply(lambda x: len(re.findall("[A-Z]", x))) / lett_count
words = names.apply(lambda x: re.findall("[a-z']+", x.lower()))
parts_of_speech = words.apply(lambda x: nltk.pos_tag(x))
just_pos = parts_of_speech.apply(lambda x: [i[1] for i in x])
pos_counts = just_pos.apply(get_count)
pos_counts.apply(lambda x: len(x)).max()

# pos_types = Counter()
# for cell in pos_counts:
#     for val in cell.keys():
#         pos_types[val] += cell[val]
        
possesive_count = pos_counts.apply(lambda x: x.get('WP', 0)) / word_count
noun_count = pos_counts.apply(lambda x: x.get('NN', 0)) / word_count
adj_count = pos_counts.apply(lambda x: x.get('JJ', 0)) / word_count
verb_count = pos_counts.apply(lambda x: x.get('VB', 0)) / word_count
preposition_count = pos_counts.apply(lambda x: x.get('IN', 0)) / word_count
determinator_count = pos_counts.apply(lambda x: x.get('DT', 0)) / word_count

df['name$word_count'] = word_count
df['name$punc_count'] = punc_count
df['name$caps_count'] = caps_count
df['name$possesive_count'] = possesive_count
df['name$noun_count'] = noun_count
df['name$adj_count'] = adj_count
df['name$verb_count'] = verb_count
df['name$preposition_count'] = preposition_count
df['name$determinator_count'] = determinator_count

keep_cols = [x for x in df.columns if 'Unnamed' not in x]
df = df[keep_cols]

df.to_csv('data/data_with_pos.csv')

In [66]:
### Add dummy cols
ndf = pd.read_csv('data/data_with_pos.csv')
for cat in df['category'].unique():
    ndf['category&{}'.format(cat)] = df['category'] == cat
for cat in df['main_category'].unique():
    ndf['main_category&{}'.format(cat)] = df['main_category'] == cat
for cat in df['country'].unique():
    ndf['country&{}'.format(cat)] = df['country'] == cat
for cat in df['currency'].unique():
    ndf['currency&{}'.format(cat)] = df['currency'] == cat
for cat in df['country'].unique():
    ndf['country&{}'.format(cat)] = df['country'] == cat    
ndf.drop(['category', 'main_category', 'country', 'currency', 'name', 'country'], axis=1, inplace=True)

ndf['deadline'] = df.loc[:,'deadline'].apply(pd.Timestamp)
ndf['launched'] = df.loc[:,'launched'].apply(pd.Timestamp)

ndf['deadline_month'] = ndf.loc[:,'deadline'].apply(lambda x: x.strftime('%b'))
ndf['launched_month'] = ndf.loc[:,'launched'].apply(lambda x: x.strftime('%b'))

ndf['launched_year'] = ndf.loc[:,'launched'].apply(lambda x: x.strftime('%Y'))

ndf['length'] = (ndf['deadline'] - ndf['launched']).apply(lambda x: x.days)

for cat in ndf['deadline_month'].unique():
    ndf['deadline_month&{}'.format(cat)] = ndf['deadline_month'] == cat
    
for cat in ndf['launched_month'].unique():
    ndf['launched_month&{}'.format(cat)] = ndf['deadline_month'] == cat

ndf.drop(['launched_month',
          'launched_year',
          'deadline_month',
          'launched',
          'deadline'], axis=1, inplace=True)

ndf.to_csv('data/data_with_dummies.csv')

In [67]:
### create binary classification dataset
ndf = pd.read_csv('data/data_with_dummies.csv')
ndf['success'] = (ndf['state'] == 'successful') | (ndf['pledged'] > ndf['goal'])
ndf.drop('state', axis=1, inplace=True)
keep_cols = [x for x in ndf.columns if 'Unnamed' not in x]
ndf = ndf[keep_cols]
ndf.to_csv('data/data_with_dummies_and_class.csv')